In [1]:
# %%
import pandas as pd
#import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
#import plotly.graph_objects as go
# 출력 제한 해제
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
pd.set_option('display.max_seq_items', None)
pd.options.display.max_colwidth = 100

import warnings
warnings.filterwarnings(action='ignore') 

In [5]:
# %%
train = pd.read_csv('../data/train.csv')
test =  pd.read_csv('../data/test.csv')
trade = pd.read_csv('../data/international_trade.csv')

train.head()


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [6]:
# %%
time = pd.to_datetime(train['timestamp'].copy())

train['Date'] = pd.to_datetime(train['timestamp'])
train['week'] = train['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
train['day_name'] = train['Date'].dt.day_name()

train['year'] = train['timestamp'].apply(lambda x : int(x[0:4]))
train['month'] = train['timestamp'].apply(lambda x : int(x[5:7]))
train['day'] = train['timestamp'].apply(lambda x : int(x[8:10]))

In [7]:
# %%
import datetime
import holidays

# %%
#train = train[(train['supply(kg)'] != 0) & (train['supply(kg)'] != 0)] # 이거차이도 큼!!! 거의 rmse 1200에서 680으로 줄어듦. 근데 이건 validation에만이고 test는 이거하면 정반대로됨 ㅠ...

# %%
#시계열 특성을 학습에 반영하기 위해 timestamp를 월, 일, 시간으로 나눕니다
test_re = test.copy()

test_re['year'] = test_re['timestamp'].apply(lambda x : int(x[0:4]))
test_re['month'] = test_re['timestamp'].apply(lambda x : int(x[5:7]))
test_re['day'] = test_re['timestamp'].apply(lambda x : int(x[8:10]))
test_re['Date'] = pd.to_datetime(test_re['timestamp'])
test_re['week'] = test_re['Date'].apply(lambda x: x.isocalendar()[1]) # 일요일 제거를 위함
test_re['day_name'] = test_re['Date'].dt.day_name()


# %%
test_re.head()

,ID,timestamp,item,corporation,location,year,month,day,Date,week,day_name
0,TG_A_J_20230304,2023-03-04,TG,A,J,2023,3,4,2023-03-04,9,Saturday
1,TG_A_J_20230305,2023-03-05,TG,A,J,2023,3,5,2023-03-05,9,Sunday
2,TG_A_J_20230306,2023-03-06,TG,A,J,2023,3,6,2023-03-06,10,Monday
3,TG_A_J_20230307,2023-03-07,TG,A,J,2023,3,7,2023-03-07,10,Tuesday
4,TG_A_J_20230308,2023-03-08,TG,A,J,2023,3,8,2023-03-08,10,Wednesday


In [8]:
# %%
from sklearn.preprocessing import LabelEncoder
train_x = train.drop(columns=['ID', 'timestamp', 'supply(kg)', 'price(원/kg)','Date'])
train_y = train['price(원/kg)']
test_x = test_re.drop(columns=['ID', 'timestamp','Date'])
qual_col = ['item', 'corporation', 'location','day_name']

for i in qual_col:
    le = LabelEncoder()
    train_x[i]=le.fit_transform(train_x[i])
    test_x[i]=le.transform(test_x[i]) #test 데이터에 대해서 fit하는 것은 data leakage에 해당합니다

In [9]:
train_x = pd.concat([train_x, train_y], axis=1)

In [10]:
train_x.head()

,item,corporation,location,week,day_name,year,month,day,price(원/kg)
0,4,0,0,1,5,2019,1,1,0.0
1,4,0,0,1,6,2019,1,2,0.0
2,4,0,0,1,4,2019,1,3,1728.0
3,4,0,0,1,0,2019,1,4,1408.0
4,4,0,0,1,2,2019,1,5,1250.0


In [11]:
test_x.head() #day_name 일치

,item,corporation,location,year,month,day,week,day_name
0,4,0,0,2023,3,4,9,2
1,4,0,0,2023,3,5,9,3
2,4,0,0,2023,3,6,10,1
3,4,0,0,2023,3,7,10,5
4,4,0,0,2023,3,8,10,6


In [12]:
train_x.item.unique()

array([4, 2, 1, 3, 0])

In [13]:
before_num = 9999
l_train_info = []
l=0
for num in train_x.item:
    if before_num != num:
        before_num = num
        l_train_info.append(l)
        print(num)
    l+=1
l_train_info.append(l)

4
2
1
3
0
1
3


In [15]:
l_train_info

[0, 15230, 25891, 31983, 42644, 56351, 57874, 59397]

In [14]:
train_x.iloc[l_train_info[7]:l_train_info[8]]

IndexError: list index out of range

In [14]:
before_num = 9999
l_info = []
l = 0
for num in test_x.item:
    if before_num != num:
        before_num = num
        print(num)
        l_info.append(l)
        l = 0
    l+=1
l_info.append(l)

4
2
1
3
0
1
3


In [22]:
from pycaret.time_series import *

preds = []
for i in range(len(l_train_info)-1):
    # 각 반복에 대한 훈련 데이터 설정
    train_data = train_x.iloc[l_train_info[i]:l_train_info[i+1]]
    fh = l_info[i+1]

    # PyCaret 설정
    s = setup(train_data, target='price(원/kg)', session_id=41, fh=fh, fold=5)

    # 여러 모델 비교하여 최적의 모델 선택 et_cds_dt
    #best = compare_models()
    
    model = create_model('et_cds_dt')
    tuned_clf = tune_model(model, optimize = 'RMSE')
    # 예측 수행
    pred = predict_model(tuned_clf, fh=fh)
    preds.extend(pred['y_pred'])


,Description,Value
0,session_id,41
1,Target,price(원/kg)
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(15230, 9)"
5,Transformed data shape,"(15230, 9)"
6,Transformed train set shape,"(14950, 9)"
7,Transformed test set shape,"(280, 9)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,13549.0000,0.9047,0.7159,1070.5182,1624.0172,1623034976544418304.0000,0.7800,0.1712
1,13829.0000,1.5875,1.1370,1877.8717,2578.9460,598443139576332032.0000,0.8473,0.0104
2,14109.0000,0.8012,0.5899,943.7239,1333.7234,557824009607177152.0000,0.5434,0.6761
3,14389.0000,0.9006,0.8226,1056.5245,1850.2576,1636333313872919296.0000,0.6269,0.5035
4,14669.0000,0.7604,0.5876,896.4842,1325.6919,1049237161675575424.0000,0.5517,0.5726
Mean,nan,0.9909,0.7706,1169.0245,1742.5272,1092974520255284352.0000,0.6699,0.3868
SD,nan,0.3035,0.2030,360.5170,461.7617,470969888205551232.0000,0.1228,0.2530


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,13549.0000,1.0194,0.7659,1206.3196,1737.5211,2567651256365086208.0000,0.7332,0.0513
1,13829.0000,1.5993,1.0809,1891.7844,2451.5157,1742232058345012736.0000,0.7443,0.1057
2,14109.0000,1.8745,1.1067,2207.8830,2502.0504,1876541983121596672.0000,0.8095,-0.1399
3,14389.0000,1.6075,1.1075,1885.8263,2491.2066,2469458062570700288.0000,0.7953,0.1000
4,14669.0000,1.2106,0.8056,1427.1411,1817.6033,1649116881923596544.0000,0.6620,0.1965
Mean,nan,1.4622,0.9733,1723.7909,2199.9794,2061000048465198080.0000,0.7489,0.0627
SD,nan,0.3063,0.1539,359.0543,346.2398,381790242942202432.0000,0.0522,0.1117


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 11.6min finished


...

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,24714.0000,1.1733,0.6294,186.4416,289.7590,519946935655676480.0000,1.9317,0.0834
1,24910.0000,1.2280,0.5622,195.3590,258.3905,829718236123799680.0000,1.9839,-2.3739
2,25106.0000,1.6780,0.6010,262.6144,273.9334,1182710288815493120.0000,2.0000,0.0000
3,25302.0000,1.3328,0.5994,204.6797,270.6036,892798093371768192.0000,1.9894,-6.6454
4,25498.0000,0.1979,0.1910,29.8498,85.4505,98792129801366960.0000,1.9997,-0.1893
Mean,nan,1.1220,0.5166,175.7889,235.6274,704793136753620864.0000,1.9809,-1.8250
SD,nan,0.4942,0.1642,77.6763,75.7525,368988424449347392.0000,0.0254,2.5763


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,20:56:42
Status,. . . . . . . . . . . . . . . . . .,Compiling Final Results
Estimator,. . . . . . . . . . . . . . . . . .,ExtraTreesRegressor


Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.8min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  6.5min finished


KeyboardInterrupt: 

In [23]:
preds

[6722.9073,
 15.6954,
 6654.7933,
 6928.3413,
 5766.077,
 5653.21,
 5861.5922,
 4632.7222,
 51.8722,
 5882.8784,
 5300.6093,
 5661.6287,
 5408.9311,
 5732.2136,
 4992.9813,
 116.2086,
 5633.5946,
 5439.8167,
 5643.0919,
 5785.3035,
 5807.311,
 5002.0677,
 244.0716,
 5746.6303,
 5426.02,
 5212.0758,
 5671.8801,
 5879.7458,
 5048.2363,
 450.6733,
 5509.0371,
 5256.7283,
 5134.4766,
 5343.8024,
 5561.3918,
 4591.9293,
 853.5713,
 5391.3811,
 5190.5237,
 4784.0584,
 5204.3858,
 5530.4851,
 4883.8989,
 1243.3259,
 5362.6248,
 5196.3381,
 4940.9151,
 5268.5809,
 5576.7125,
 4963.1539,
 1458.4197,
 5267.9892,
 5245.154,
 5068.1315,
 5358.2496,
 5471.4353,
 4824.0659,
 1781.9608,
 5212.0928,
 5064.626,
 5005.56,
 5298.3473,
 5658.5703,
 5091.7702,
 2221.9837,
 5287.6169,
 5538.8513,
 5203.9984,
 5299.7994,
 5427.3268,
 3077.6431,
 2405.0197,
 5089.2703,
 4864.9332,
 4699.6013,
 5040.5503,
 5137.8601,
 4057.0664,
 2463.1225,
 5116.7433,
 5022.7728,
 4837.7325,
 5012.1889,
 5144.3323,
 4923.0525

In [86]:
tuned_clf = tune_model(llar, optimize = 'RMSE')
# predictions = predict_model(tuned_clf, data = test_x)

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,59088.0000,0.9500,0.5234,74.0714,97.2093,0.1325,0.1386,0.7846
1,59116.0000,1.5375,0.9199,120.0357,169.5834,0.1509,0.1728,0.6230
2,59144.0000,5.2939,3.0162,414.1786,552.4482,1474768035120002816.0000,0.8915,-0.6082
3,59172.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
4,59200.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
5,59228.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0000
6,59256.0000,0.2843,0.6254,21.9643,116.2241,0.0357,0.0714,-0.0370
7,59284.0000,3.7187,2.2070,283.2143,407.6237,98918348958316256.0000,1.0046,-0.9246
8,59312.0000,1.5394,0.9366,121.3929,176.3356,94736435018615072.0000,0.2646,0.3485
9,59340.0000,1.7406,1.1441,138.1429,214.8353,90554521078913904.0000,0.4111,0.0742


Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 out of  40 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.3s finished


In [26]:
predictions = predict_model(llar, fh=fh)

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,Seasonal Naive Forecaster,0.4870,0.2386,221.5568,299.1455,708269400738186368.0000,0.7930,-0.2788


In [93]:
preds

[3053.4527,
 3033.8032,
 3229.47,
 3314.3951,
 3075.088,
 3344.8296,
 3230.6479,
 2909.9569,
 3046.3071,
 3223.2303,
 3371.2762,
 3016.8858,
 3341.1264,
 3328.2931,
 3053.4527,
 3033.8032,
 3229.47,
 3314.3951,
 3075.088,
 3344.8296,
 3230.6479,
 2909.9569,
 3046.3071,
 3223.2303,
 3371.2762,
 3016.8858,
 3341.1264,
 3328.2931,
 3053.4527,
 3033.8032,
 3229.47,
 3314.3951,
 3075.088,
 3344.8296,
 3230.6479,
 2909.9569,
 3046.3071,
 3223.2303,
 3371.2762,
 3016.8858,
 3341.1264,
 3328.2931,
 3053.4527,
 3033.8032,
 3229.47,
 3314.3951,
 3075.088,
 3344.8296,
 3230.6479,
 2909.9569,
 3046.3071,
 3223.2303,
 3371.2762,
 3016.8858,
 3341.1264,
 3328.2931,
 3053.4527,
 3033.8032,
 3229.47,
 3314.3951,
 3075.088,
 3344.8296,
 3230.6479,
 2909.9569,
 3046.3071,
 3223.2303,
 3371.2762,
 3016.8858,
 3341.1264,
 3328.2931,
 3053.4527,
 3033.8032,
 3229.47,
 3314.3951,
 3075.088,
 3344.8296,
 3230.6479,
 2909.9569,
 3046.3071,
 3223.2303,
 3371.2762,
 3016.8858,
 3341.1264,
 3328.2931,
 3053.4527

In [22]:
tuned_clf = tune_model(llar, optimize = 'RMSE')
predictions = predict_model(llar, data = test_x)

SyntaxError: invalid syntax (3477283352.py, line 1)

In [1]:
result = pd.concat([test_re, predictions['prediction_label']], axis = 1)

result['answer'] = result['prediction_label'].copy()

del result['timestamp']
del result['item']
del result['corporation']
del result['location']
del result['year']
del result['month']
del result['day']
del result['Date']
del result['week']
del result['day_name']
del result['prediction_label']

NameError: name 'pd' is not defined

In [14]:
for i in range(len(result)):
    if (i-1) % 7 == 0: # 1, 8, 15, 22
        result.iloc[i,-1] = 0

result.head(20)

,ID,answer
0,TG_A_J_20230304,3875.56
1,TG_A_J_20230305,0.00
2,TG_A_J_20230306,3454.97
3,TG_A_J_20230307,3271.77
4,TG_A_J_20230308,3452.73
5,TG_A_J_20230309,3904.98
6,TG_A_J_20230310,3639.50
7,TG_A_J_20230311,3878.27
8,TG_A_J_20230312,0.00
9,TG_A_J_20230313,3699.92


In [15]:
result.to_csv('result1_tuned.csv', index = False)